### --- Step 1: Imports ---

In [8]:
from functions.llm_chain import generate_response_openai

Workshop path: /Users/rcnnnghm/work/hopsworks/workshop


In [1]:
import hopsworks
import exclude.key
from openai import OpenAI
import os
import getpass
import joblib

# Hopsworks
HOPSWORKS_API_KEY = exclude.key.HOPSWORKS_API_KEY
OPENAI_API_KEY = exclude.key.OPENAI_API_KEY

FEATURE_GROUP_NAME = "bars_near_london_bridge"
FEATURE_VIEW_NAME = "busyness_feature_view"
FEATURE_GROUP_VERSION = 3
FEATURE_VIEW_VERSION = 4

project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/25749
Connected. Call `.close()` to terminate connection gracefully.


In [2]:
fg = fs.get_feature_group(
    name=FEATURE_GROUP_NAME, 
    version=FEATURE_GROUP_VERSION,
)
fg

## -- Step 2 Retrieve Feature View -- 

In [3]:
# Retrieve the 'busyness_feature_view'
feature_view = fs.get_feature_view(
    name=FEATURE_VIEW_NAME,
    version=FEATURE_VIEW_VERSION,
)

# Initialize batch scoring
feature_view.init_batch_scoring(1)
feature_view

In [4]:
# Retrieve the model registry
mr = project.get_model_registry()

# Retrieve the 'busyness_xgboost_model' from the model registry
retrieved_model = mr.get_model(
    name="xgboost_busyness",
    version=3,
)
retrieved_model

Connected. Call `.close()` to terminate connection gracefully.


Model(name: 'xgboost_busyness', version: 3)

In [5]:
model_dir = "busyness_model"

# Retrieve the model registry
mr = project.get_model_registry()

# Retrieve the 'busyness_xgboost_model' from the model registry
retrieved_model = mr.get_model(
    name="xgboost_busyness",
    version=3,
)

# Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()
saved_model_dir = "./"

# Load the XGBoost regressor model and label encoder from the saved model directory
model_busyness = joblib.load(saved_model_dir + f"{model_dir}/xgboost_busyness.pkl")
# encoder = joblib.load(saved_model_dir + "/label_encoder.pkl")

# Display the retrieved XGBoost regressor model
model_busyness


Connected. Call `.close()` to terminate connection gracefully.


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

## -- Step 3 Ask the Question! -- 

In [6]:

client = OpenAI(
    api_key=OPENAI_API_KEY,
)

In [9]:
QUESTION = "What will be the best bar to go in an hour from now next to London Bridge?"

response = generate_response_openai(   
    QUESTION,
    feature_view,
    model_busyness,
    client,
    verbose=True,
)
print(response)

2024-06-12 14:50:57,776 INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
🗓️ Today's date: Wednesday, 2024-06-12
📖 
2024-06-12 14:51:04,117 INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
The best bar to visit near London Bridge an hour from now would be The Hops & Glory. It's known for its cozy atmosphere and extensive selection of craft beers. On Wednesdays, they usually have a live jazz band playing in the evening, which adds to the relaxed, vibrant atmosphere. The bar also features an outdoor seating area, great for enjoying the evening weather. It's quite popular among locals and visitors alike, so it might be busy, but the experience is well worth it.
